In [1]:
import os, sys, time
import sqlite3
import pickle as pkl
import numpy as np

In [2]:
data_dir = 'data'
fdb = os.path.join(data_dir, 'msd/track_metadata.db')
faotm = os.path.join(data_dir, 'aotm-2011/aotm-2011-subset.pkl')

Load (a subset of) AotM playlist data.

In [3]:
playlists = pkl.load(open(faotm, 'rb'))

In [4]:
len(playlists)

241

Connect the SQLite MSD metadata DB.

In [5]:
conn = sqlite3.connect(fdb)

Get table names.

In [6]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table'")
data = res.fetchall()
conn.commit()

In [7]:
data

[('songs',)]

Get column names.

In [8]:
res = conn.execute('PRAGMA table_info(songs)')
data = res.fetchall()
conn.commit()

In [9]:
data

[(0, 'track_id', 'text', 0, None, 1),
 (1, 'title', 'text', 0, None, 0),
 (2, 'song_id', 'text', 0, None, 0),
 (3, 'release', 'text', 0, None, 0),
 (4, 'artist_id', 'text', 0, None, 0),
 (5, 'artist_mbid', 'text', 0, None, 0),
 (6, 'artist_name', 'text', 0, None, 0),
 (7, 'duration', 'real', 0, None, 0),
 (8, 'artist_familiarity', 'real', 0, None, 0),
 (9, 'artist_hotttnesss', 'real', 0, None, 0),
 (10, 'year', 'int', 0, None, 0)]

Map `song_id` to `track_id`.

In [10]:
playlists[100]['filtered_lists']

[['SOOAUWM12A8C134D59', 'SORFJHR12A67AD87DC']]

In [11]:
song_id_set = {sid for playlist in playlists for sid in playlist['filtered_lists'][0]}

In [12]:
len(song_id_set)

1718

In [13]:
map_song_track = dict()  # song_id --> track_id

In [14]:
for sid in sorted(song_id_set):
    res = conn.execute("SELECT track_id FROM songs WHERE song_id='" + sid + "'")
    data = res.fetchall()
    conn.commit()
    tids = [t[0] for t in data]
    map_song_track[sid] = tids

In [15]:
len(map_song_track)

1718

In [18]:
#map_song_track

Close SQLite DB connection.

In [19]:
conn.close()

Save mapping.

In [20]:
pkl.dump(map_song_track, open(os.path.join(data_dir, 'aotm-2011/map_song_track.pkl'), 'wb'))